In [12]:
import os
from PIL import Image
from shutil import copy2

In [13]:
dataset_base_path = "../og_dataset/"
mask_class_path = os.path.join(dataset_base_path, "classes/masks/")
image_base_path = os.path.join(dataset_base_path, "classes/jpg/")
trainset_file_names_path = os.path.join(dataset_base_path, "trainset.filenames.txt")
testset_file_names_path = os.path.join(dataset_base_path, "testset.filenames.txt")
testset_nologo_files_path = os.path.join(dataset_base_path, "testset-nologos.relpaths.txt")
valset_file_names_path = os.path.join(dataset_base_path, "valset.filenames.txt")
valset_nologo_files_path = os.path.join(dataset_base_path, "valset-nologos.relpaths.txt")
yolo_output_base_path = "../yolo_dataset/"
rcnn_output_base_path = "../rcnn_dataset/"


In [14]:
def convert_to_yolo_format(image_width, image_height, bbox_x1, bbox_y1, bbox_width, bbox_height):
    """
    Function to convert the Flickr-32 bounding box annotation to the YOLO format
    """
    bbox_x2 = bbox_x1 + bbox_width
    bbox_y2 = bbox_y1 + bbox_height
    bbox_cx = (bbox_x1 + bbox_x2) // 2
    bbox_cy = (bbox_y1 + bbox_y2) // 2

    yolo_x = bbox_cx / image_width
    yolo_y = bbox_cy / image_height
    yolo_width = bbox_width / image_width
    yolo_height = bbox_height / image_height
    
    return(yolo_x, yolo_y, yolo_width, yolo_height)

In [15]:
def convert_to_rcnn_format(bbox_x1, bbox_y1, bbox_width, bbox_height):
    """
    Function to convert the Flickr-32 bounding box annotation to the R-CNN format
    """
    bbox_x2 = bbox_x1 + bbox_width
    bbox_y2 = bbox_y1 + bbox_height
    return(bbox_x1, bbox_y1, bbox_x2, bbox_y2)

In [16]:
def get_bb_annotations(text_file_path):
    """
    Function to get the bounding box annotations for each file in the Flickr-32 dataset
    """
    bb_annotations = []
    with open(text_file_path) as f:
        lines = f.readlines()
        for line in lines[1:]:
            bbox_x1, bbox_y1, bbox_width, bbox_height = line.split(" ")
            bb_annotations.append([int(bbox_x1), int(bbox_y1), int(bbox_width), int(bbox_height)])
        return bb_annotations

In [17]:
# Forward and Reverse Lookup Table for Class Label <-> Class Name
class_names = next(os.walk(mask_class_path))[1]
label_class_dict = {index + 1 : class_name for index, class_name in enumerate(class_names)}
class_label_dict = {class_name : index + 1 for index, class_name in enumerate(class_names)}
print(label_class_dict)
print(class_label_dict)

{1: 'adidas', 2: 'aldi', 3: 'apple', 4: 'becks', 5: 'bmw', 6: 'carlsberg', 7: 'chimay', 8: 'cocacola', 9: 'corona', 10: 'dhl', 11: 'erdinger', 12: 'esso', 13: 'fedex', 14: 'ferrari', 15: 'ford', 16: 'fosters', 17: 'google', 18: 'guiness', 19: 'heineken', 20: 'hp', 21: 'milka', 22: 'nvidia', 23: 'paulaner', 24: 'pepsi', 25: 'rittersport', 26: 'shell', 27: 'singha', 28: 'starbucks', 29: 'stellaartois', 30: 'texaco', 31: 'tsingtao', 32: 'ups'}
{'adidas': 1, 'aldi': 2, 'apple': 3, 'becks': 4, 'bmw': 5, 'carlsberg': 6, 'chimay': 7, 'cocacola': 8, 'corona': 9, 'dhl': 10, 'erdinger': 11, 'esso': 12, 'fedex': 13, 'ferrari': 14, 'ford': 15, 'fosters': 16, 'google': 17, 'guiness': 18, 'heineken': 19, 'hp': 20, 'milka': 21, 'nvidia': 22, 'paulaner': 23, 'pepsi': 24, 'rittersport': 25, 'shell': 26, 'singha': 27, 'starbucks': 28, 'stellaartois': 29, 'texaco': 30, 'tsingtao': 31, 'ups': 32}


In [18]:
def get_id_set(file_path):
    """
    Function to get a set of file identifiers belonging to a particular split (train / validaion / test)
    """
    with open(file_path) as f:
        lines = f.readlines()
    file_id_set = {line.split(".")[0] for line in lines}
    return file_id_set

In [19]:
trainset_file_ids = get_id_set(trainset_file_names_path)
valset_file_ids = get_id_set(valset_file_names_path)
testset_file_ids = get_id_set(testset_file_names_path)

In [20]:
def get_save_path(file_identifier, annotation_format):
    """
    Function to get the save path of a file
    """
    base_save_path = yolo_output_base_path if annotation_format == "YOLO" else rcnn_output_base_path
    if file_identifier in trainset_file_ids:
        return os.path.join(base_save_path, "train", "")
    elif file_identifier in valset_file_ids:
        return os.path.join(base_save_path, "validation", "")
    elif file_identifier in testset_file_ids:
        return os.path.join(base_save_path, "test", "")


In [21]:
def save_file(image_file_path, file_identifier, class_label, annotations, annotation_format):
    """
    Function to copy the image and it's YOLO annotations to the correct split location (train / validate / test)
    """
    save_path = get_save_path(file_identifier, annotation_format)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    copy2(image_file_path, save_path)
    annotation_file_path = os.path.join(save_path, file_identifier + ".txt")
    with open(annotation_file_path, "w") as f:
        for annotation in annotations:
            if annotation_format == "YOLO":
                annotation_list = [1]
                annotation_list.extend(annotation)
            elif annotation_format == "R-CNN":
                annotation_list = list(annotation)
                annotation_list.append(1)

            annotation_list = [str(item) for item in annotation_list]
            annotation_line = " ".join(annotation_list)
            annotation_line += "\n"
            f.write(annotation_line)

In [22]:
for root, dirs, files in os.walk(mask_class_path):
    for file in files:
        if file.endswith(".txt"):
            class_name = root.split("/")[-1]
            class_label = class_label_dict[class_name]

            bb_annotation_file_path = os.path.join(root, file)
            file_identifier = file.split(".")[0]
            
            image_file_path = os.path.join(image_base_path, class_name, file_identifier + ".jpg")
            image_width, image_height = Image.open(image_file_path).size
            bb_annotations = get_bb_annotations(bb_annotation_file_path)
            yolo_annotations = []
            rcnn_annotations = []
            for bb_annotation in bb_annotations: 
                bbox_x1, bbox_y1, bbox_width, bbox_height = bb_annotation
                yolo_annotations.append(convert_to_yolo_format(image_width, image_height, bbox_x1, bbox_y1, bbox_width, bbox_height))
                rcnn_annotations.append(convert_to_rcnn_format(bbox_x1, bbox_y1, bbox_width, bbox_height))

            save_file(image_file_path, file_identifier, class_label, yolo_annotations, "YOLO")
            save_file(image_file_path, file_identifier, class_label, rcnn_annotations, "R-CNN")

In [23]:
# Add the No Logo files to the respective split
with open(os.path.join(valset_nologo_files_path)) as f:
    valset_no_logo_paths = [os.path.join(dataset_base_path, line.strip("\n")) for line in f.readlines()]
for file_path in valset_no_logo_paths:
    copy2(file_path, os.path.join(yolo_output_base_path, "validation", ""))
    copy2(file_path, os.path.join(rcnn_output_base_path, "validation", ""))

with open(os.path.join(testset_nologo_files_path)) as f:
    testset_no_logo_paths = [os.path.join(dataset_base_path, line.strip("\n")) for line in f.readlines()]
for file_path in testset_no_logo_paths:
    copy2(file_path, os.path.join(yolo_output_base_path, "test", ""))
    copy2(file_path, os.path.join(rcnn_output_base_path, "test", ""))
